In [0]:
import pyspark
from  pyspark.sql.functions import concat_ws
import os

In [0]:
import collections
import pyspark

df = spark.table("default.grade_2_txt")

'''df_text = df.select(concat_ws(",", *df.columns).alias("value"))
df_text.write.mode("overwrite").text("dbfs:/tmp/my_table_text")
dbutils.fs.ls("dbfs:/tmp/my_table_text")
df_readback = spark.read.text("dbfs:/tmp/my_table_text").rdd
'''

print(type(df))
print(df)

df = df.rdd.map(lambda row: row['_c0'])
print(df.collect()) # tom 70
df = df.map(lambda line: line.split(" ")[1])
print(df.collect()) # 70. collect() returns 'list' type
df_cnt = df.countByValue()
print(df_cnt) # countByValue has made a 'dictionary type'



for grade, count in sorted(df_cnt.items(), key=lambda item: item[1], reverse=True):
# grade, count are the names of each column
# items() function is going to return a list of a dictionary 'key-value' pairs
# and is also going throught the loop with both keys and values
    print(f"{grade}: {count}")
    

# grade_cnt = grade.countByValue()

<class 'pyspark.sql.dataframe.DataFrame'>
DataFrame[_c0: string]
['tom 70', 'sara 80', 'joon 100', 'kevin 90', 'John 90']
['70', '80', '100', '90', '90']
defaultdict(<class 'int'>, {'70': 1, '80': 1, '100': 1, '90': 2})
90: 2
70: 1
80: 1
100: 1


In [0]:
sc = pyspark.SparkContext.getOrCreate();
# Key / Value RDD

# creating Key / Value RDD
# total_by_brand = rdd.map(lambda brand: (brand, 1))

# reduceByKey(): Merge the values for each key using an associative and commutative reduce function.
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())
# [('a', 2), ('b', 1)]

Out[53]: [('a', 2), ('b', 1)]

In [0]:
# default.house_price_csv
data = spark.table("default.house_price_csv") # dataframe
data.show()
data = data.withColumnRenamed("_c0", "city").withColumnRenamed("_c1", "price").withColumnRenamed("_c2", "count") # giving names to columns
data.show() # still dataframe

# have to manually import col because 'import pyspark' only imports the top level
from pyspark.sql.functions import col, when, lit

parsed_data = data.select(
    # selecting wanted columns from a dataframe
    #col("city"),
    col("price").cast("int"),
    col("count").cast("int")
)

print(parsed_data.collect())
# [Row(price=10000, count=3), Row(price=10000, count=5), Row(price=40000, count=7), Row(price=5000, count=7), Row(price=4000, count=2), Row(price=9000, count=4), Row(price=5000, count=7), Row(price=4000, count=2), Row(price=8000, count=9)]
print("------------------")

rdd = data.select("price", "count").rdd.map(lambda row: (row["price"], row["count"])) # making dataframe -> rdd format data
# print(rdd.collect()) # list of tuples finally

result = rdd.mapValues(lambda count: (count, 1)).reduceByKey(lambda a, b: (int(a[0]) + int(b[0]), int(a[1]) + int(b[1])))
# reduceByKey is 'grouping' by key and calculae(values)

print("\n\n\n\n\n\n")
print(result.collect())
# [('10000', (8, 2)), ('40000', ('7', 1)), ('5000', (14, 2)), ('4000', (4, 2)), ('9000', ('4', 1)), ('8000', ('9', 1))]

sum_of_count = result.mapValues(lambda total_count: int(total_count[0]) / total_count[1])

print(sum_of_count.collect())
# [('10000', 4.0), ('40000', 7.0), ('5000', 7.0), ('4000', 2.0), ('9000', 4.0), ('8000', 9.0)]

print("\n\n\n\n\n\n")
# result_rdd = rdd.mapValues(lambda count: count())


avg_by_count = sum_of_count.mapValues(lambda count: (count, 1)).reduceByKey(lambda a, b: (int(a[0]) + int(b[0]), int(a[1]) + int(b[1])))

print(avg_by_count.collect())
# [('10000', (4.0, 1)), ('40000', (7.0, 1)), ('5000', (7.0, 1)), ('4000', (2.0, 1)), ('9000', (4.0, 1)), ('8000', (9.0, 1))]


'''avg_by_count = sum_of_count.mapValues(lambda total_count: int(total_count[0]) / total_count[1])
results = avg_by_count.collect()
print(type(result))
'''
# print(price_cnt.collect())

# to print RDD
# print(price_cnt.first())




+-------+-----+---+
|    _c0|  _c1|_c2|
+-------+-----+---+
|  seoul|10000|  3|
|  seoul|10000|  5|
|  seoul|40000|  7|
|  busan| 5000|  7|
|incheon| 4000|  2|
|  busan| 9000|  4|
|incheon| 5000|  7|
|  seoul| 4000|  2|
|  busan| 8000|  9|
+-------+-----+---+

+-------+-----+-----+
|   city|price|count|
+-------+-----+-----+
|  seoul|10000|    3|
|  seoul|10000|    5|
|  seoul|40000|    7|
|  busan| 5000|    7|
|incheon| 4000|    2|
|  busan| 9000|    4|
|incheon| 5000|    7|
|  seoul| 4000|    2|
|  busan| 8000|    9|
+-------+-----+-----+

[Row(price=10000, count=3), Row(price=10000, count=5), Row(price=40000, count=7), Row(price=5000, count=7), Row(price=4000, count=2), Row(price=9000, count=4), Row(price=5000, count=7), Row(price=4000, count=2), Row(price=8000, count=9)]
------------------
[('10000', '3'), ('10000', '5'), ('40000', '7'), ('5000', '7'), ('4000', '2'), ('9000', '4'), ('5000', '7'), ('4000', '2'), ('8000', '9')]







[('10000', (8, 2)), ('40000', ('7', 1)), ('5000',

In [0]:
 # practice 3

df = spark.table("default.temperature_1_csv")
df.show()


df = df.withColumnRenamed("City", "city").withColumnRenamed("AverageTemperatureFahr", "avg_temp_fahr")

parsed_data = df.select(
    # selecting wanted columns from a dataframe
    #col("city"),
    col("city"),
    col("avg_temp_fahr").cast("int")
)

# changing the dataframe into rdd
rdd = df.select("city", "avg_temp_fahr")
rdd = df.select("city", "avg_temp_fahr").rdd.map(lambda row: (row["city"], row["avg_temp_fahr"]))
# print(rdd.collect())
# filter "NA" value
rdd = rdd.filter(lambda x: "NA" not in x[1])

# min temperature
min_temp = rdd.reduceByKey(lambda x, y: min(float(x), float(y))) # comparing multiple values with one specific key
final_list = min_temp.collect()
print(final_list)

filtered_final_list = [(city, float(temperature)) for city, temperature in final_list if city!= 'NA']

for city, temperature in filtered_final_list:
    print(f"{city}: {temperature}")


+---------+-----+---+----+----------------------+---------------------------------+--------+----------+-----------+--------+---------+
|record_id|month|day|year|AverageTemperatureFahr|AverageTemperatureUncertaintyFahr|    City|country_id|    Country|Latitude|Longitude|
+---------+-----+---+----+----------------------+---------------------------------+--------+----------+-----------+--------+---------+
|   474376|   01| 01|1853|                    NA|                               NA|Auckland|       NEW|New Zealand|  36.17S|  175.03E|
|   474377|   02| 01|1853|                    NA|                               NA|Auckland|       NEW|New Zealand|  36.17S|  175.03E|
|   474378|   03| 01|1853|                    NA|                               NA|Auckland|       NEW|New Zealand|  36.17S|  175.03E|
|   474379|   04| 01|1853|                    NA|                               NA|Auckland|       NEW|New Zealand|  36.17S|  175.03E|
|   474380|   05| 01|1853|                    NA|      

In [0]:
# practice 4. flatMap
df = spark.table("default.lorem_ipsum_1_txt")
df.printSchema()
combined_rows = df.rdd.map(lambda row: " ".join([str(c) for c in row if c is not None]))

words_rdd = combined_rows.flatMap(lambda line: line.split())
# print(words_rdd.collect())

# count
'''word_cnt = words_rdd.countByValue() # create a dictionary
for word, count in word_cnt.items(): # items() is used with dictionaries to get a view of the dictionary's key-value pairs
    print(f"{word}: {count}")
'''

# sorting by the most frequent word
word_cnt = words_rdd.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y) # calculation on the 'values' grouping by the 'key'
# print(word_cnt.collect())
sorted_word_count = word_cnt.map(lambda x: (x[1], x[0])).sortByKey(ascending=False)
#print(sorted_word_count.collect())

for word, count in sorted_word_count.collect():
    print(f"{word}: {count}")

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)

6: the
4: Lorem
4: of
3: Ipsum
3: and
2: dummy
2: text
2: typesetting
2: has
2: a
2: type
2: It
2: with
1: is
1: simply
1: printing
1: industry.
1: been
1: industry's
1: standard
1: ever
1: since
1: 1500s
1: when
1: an
1: unknown
1: printer
1: took
1: galley
1: scrambled
1: it
1: to
1: make
1: specimen
1: book.
1: survived
1: not
1: only
1: five
1: centuries
1: but
1: also
1: leap
1: into
1: electronic
1: remaining
1: essentially
1: unchanged.
1: was
1: popularised
1: in
1: 1960s
1: release
1: Letraset
1: sheets
1: containing
1: passages
1: more
1: recently
1: desktop
1: publishing
1: software
1: like
1: Aldus
1: PageMaker
1: including
1: versions
1: Ipsum.


In [0]:
# practice 5. DataFrame
from pyspark.sql import (
    Row, SparkSession
)
from pyspark.sql.functions import col, asc, desc

df = spark.table("default.income_2_txt")
# print(df.show())

df.createOrReplaceGlobalTempView("income") # making the dataframe into the global temp view table
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# 1. Spark SQL
df = spark.sql(
    "SELECT * FROM global_temp.income WHERE compensation >= 70000 AND compensation <= 100000")
    # when accessing the 'income' table, it is in the global_temp database, not the 'default' database
df.show()

# 2. DataFrame function
df.groupBy("country").count().orderBy(col("count").desc()).show()

+------------------+--------------------+--------------------+------------+
|              name|             country|               email|compensation|
+------------------+--------------------+--------------------+------------+
|  Willian Cummings|             Senegal|    areus@test.canon|       77369|
|      Clarita Gill|             Ecuador| tomaslau@test.games|       86986|
| Walter Washington|          Kazakhstan|mbilderbach@examp...|       91072|
|       Lexie Banks|                Mali|unterdreht@test.date|       97933|
|        Luise Hunt|               Kenya|adellecharles@tes...|       96175|
|     Sebrina Walsh|         Puerto Rico|andrewcohen@examp...|       99276|
|      Josiah Lyons|              Malawi|nandini_m@test.ry...|       91768|
|      Temeka Grant|              Israel|terryxlife@test.g...|       71642|
|  Narcisa Saunders|Palestinian Terri...|raquelwilson@exam...|       77287|
|      Lisbeth Lane|          Azerbaijan|coreyweb@test.coffee|       82473|
|       Evan

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    avg, col, round as rnd
)

# spark = SparkSession.builder.appName("sql_import_csv").getOrCreate()

df = spark.table("age_csv")
# typecasting the 'age' column into 'int' type
df = df.withColumn("age", col("age").cast("int"))

df.filter(df.age > 40).orderBy(col("age").desc()).show()
# orderBy(col("count").desc()

# df.groupBy("age").count().show()
# df.select(df.name, df.age, df.age - 10).show()


df.select(df.name, df.age, df.country).groupBy("country").agg(rnd(avg("age"),2).alias("avg_age")).show()#.sort("avg(age)").show()

+---------------+---+--------------------+
|           name|age|             country|
+---------------+---+--------------------+
| Lauryn Hubbard| 80|           Mauritius|
|  Verena Dennis| 80|United States of ...|
|  Janee Holland| 80|         Switzerland|
|     Dung Logan| 80|             Bahamas|
|  Jadwiga Ellis| 80|            Mongolia|
|   Josiah Evans| 80|               Niger|
|     Janay Ryan| 80|            Dominica|
|   Katia Santos| 80|             Algeria|
| Hilario Little| 80|           Singapore|
|    Daron Allen| 80|Falkland Islands ...|
| Patricia Mccoy| 80|            Pakistan|
| Sherika Jordan| 80|             Mayotte|
|Roselle Gardner| 80|             Senegal|
|  Shavonne Hart| 80|       Cote d'Ivoire|
|Svetlana Hanson| 80|             Albania|
|  Lindsay Myers| 80|       Cote d'Ivoire|
|    Samual Shaw| 79|              Tuvalu|
|    Jewell Boyd| 79|      Cayman Islands|
|    Arletha Ray| 79|                Mali|
|  Arlie Rodgers| 79| Trinidad and Tobago|
+----------